In [1]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, datasets
import torch.utils.data as data

In [2]:
import deeptriplet
import deeptriplet.models
import deeptriplet.datasets

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

True
2


## Load GrabCut dataset

In [4]:
dataset = deeptriplet.datasets.GrabCut(grabcut_root="/scratch-second/yardima/datasets/GrabCut/")

## Load trained embedding model

In [5]:
model_path_random = "/srv/glusterfs/yardima/runs/deeplabv3_plus/triplet-aug/run_8/models/deeplabv3p-triplet-aug_epoch-79.pth"
    

In [11]:
d1 = torch.load(model_path_random)
from modeling.deeplab import *

net = DeepLab(backbone='resnet', output_stride=16, num_classes=50, sync_bn=True, freeze_bn=False)
net = net.cuda()
net.load_state_dict(d1)

net = net.eval().cuda();

## Interactive plot

In [6]:
from sklearn.neighbors import KNeighborsClassifier

In [7]:
class InteractiveSegmentation:

    def __init__(self, img, lbl, embed):
        fig = plt.figure(figsize=(6,6))
        ax = fig.add_subplot(111)
        ax.imshow(np.transpose(img.squeeze() / 4 + 0.45, axes=[1, 2, 0]))
        # ax.plot(np.random.rand(10))
        text=ax.text(0,0, "", va="bottom", ha="left")

        self.text = text
        
        self.samples_positive = []
        self.samples_negative = []
        
        self.ax = ax
        self.fig = fig
        
        self.img = img
        self.lbl = lbl
        self.embed = embed
        
        cid = fig.canvas.mpl_connect('button_press_event', lambda x: self.onclick(x))
        
        self.fg_color = np.array([0.1, 0.2, 1., 0.6])
        self.bg_color = np.array([0.4, 0.0, 0.0, 0.0])
        
        self.mask = np.zeros(shape=(lbl.shape[0], lbl.shape[1], 4), dtype=np.float32)
        self.maskim = ax.imshow(self.mask)
        
        self.h = int(lbl.shape[0])
        self.w = int(lbl.shape[1])
        
        
    def update_plot(self):
        
        knn = KNeighborsClassifier(n_neighbors=1)
        
        ixp = [i[1] * self.w + i[0] for i in self.samples_positive]
        ixn = [i[1] * self.w + i[0] for i in self.samples_negative]
        
        X = self.embed.reshape(self.w * self.h, -1)[ixp + ixn, :]
        y = np.array([1] * len(ixp) + [0] * len(ixn))
        
        print(X.shape)
        print(y.shape)
        
        knn.fit(X, y)
        
        pred = knn.predict(self.embed.reshape(self.w * self.h, -1)).reshape(self.h, self.w)
        
        r = np.zeros(pred.shape)
        g = np.zeros(pred.shape)
        b = np.zeros(pred.shape)
        a = np.zeros(pred.shape)
        
        r[pred == 1] = self.fg_color[0]
        g[pred == 1] = self.fg_color[1]
        b[pred == 1] = self.fg_color[2]
        a[pred == 1] = self.fg_color[3]
        
        self.mask[:,:,0] = r
        self.mask[:,:,1] = g
        self.mask[:,:,2] = b
        self.mask[:,:,3] = a
        
        self.maskim.set_data(self.mask)
        


    def onclick(self, event):
        point = (int(round(event.xdata)), int(round(event.ydata)))

        if event.button == 1:
            self.samples_positive.append(point)
            self.samples_positive = list(set(self.samples_positive))
            self.ax.add_artist(plt.Circle(point, 2.2, color='r'))
        else:
            self.samples_negative.append(point)
            self.samples_negative = list(set(self.samples_negative))
            self.ax.add_artist(plt.Circle(point, 2.2, color='b'))
            
        if len(self.samples_negative) > 0 and len(self.samples_positive) > 0:
            self.update_plot()
        
        tx = 'button=%d, xdata=%d, ydata=%d' % (event.button, point[0], point[1])
        self.text.set_text(tx)
        
        ax.draw()

In [12]:
img, lbl = dataset[7]
img = torch.tensor(img)
lbl = torch.tensor(lbl)

with torch.no_grad():  
    embed = net.forward(img.cuda().unsqueeze(0))
    embed = embed.squeeze().permute(1, 2, 0).cpu()
    
p = InteractiveSegmentation(img.numpy(), lbl.numpy(), embed.numpy())

/home/yardima/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: name 'net' is not defined